# (Optional) 웹 크롤링2 - Dynamic Crawling

201921529 김채연

**시작하기에 앞서 기초 환경만들기! step1,2**

- step1 , 크롬 버전 확인하고 크롬 웹드라이버 다운받기
https://chromedriver.chromium.org/downloads

In [1]:
# 자신이 사용하는 해당 폴더를 확인하고 이 곳에 크롬 웹드라이버 옮기기
import os
os.getcwd()

'C:\\Users\\chaeyeon\\capstone_python'

- step2, selenium 설치하기

# 0. 라이브러리

In [2]:
# 필요한 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import time


# 1. Selenium 기초
자신의 크롬 버전을 확인하고 크롬 웹드라이버를 다운받아놓아야합니다.

- 2021.09.22 기준 최신 버전: **94.0.4606.54** -> **Chrome version 94** 다운 받으면 됨!
- https://chromedriver.chromium.org/downloads

## 1.1. Simple Text Crawling
멜론 사이트에서 노래 제목을 크롤링해보자

URL: https://www.melon.com/chart/index.htm

In [6]:
# 자신이 'chromedriver.exe'를 저장한 위치의 주소
DRIVER_PATH = 'C:/Users/chaeyeon/AppData/Roaming/Microsoft/Windows/Start Menu/Programs/Anaconda3 (64-bit)/chromedriver.exe'

In [7]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH) # 드라이버를 로드

# 웹페이지가 로딩될때까지 기다리고 10초가 넘어가면 웹페이지가 로딩이 됐던 안됐던 다음 명령어를 실행하겠다는 의미
driver.implicitly_wait(10) 

# 멜론 사이트 주소
url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source # driver.page_source:브라우저에 보이는 그대로의 HTML
soup = BeautifulSoup(html, 'html.parser')

# title crawling
# 제목과 가수의 이름이 기재된 요소 사용
# tr:nth-child(1) : 첫번째 노래 제목과 가수의 이름 크로링하기
title = WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.CSS_SELECTOR, "#frm > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > div")))

# print("Title: {}".format(title.text))

title.text

'STAY\nThe Kid LAROI, Justin Bieber'

css selector의 규칙을 찾아본다

- 1번째 제목: #frm > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > div"
- 2번째 제목: #frm > div > table > tbody > tr:nth-child(2) > td:nth-child(6) > div > div
. . .

- 100번째 제목: #frm > div > table > tbody > tr:nth-child(100) > td:nth-child(6) > div > div

또는 XPATH로도 확인해보자 (full Xpath)

- 1번째 제목: //*[@id="frm"]/div/table/tbody/tr[1]/td[6]/div/div
- 2번째 제목: //*[@id="frm"]/div/table/tbody/tr[2]/td[6]/div/div
. . .

- 50번째 제목: //*[@id="frm"]/div/table/tbody/tr[100]/td[6]/div/div

In [8]:
#  2번째 제목과 가수 크롤링하기
WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.XPATH, "//*[@id='frm']/div/table/tbody/tr[2]/td[6]/div/div"))).text

'신호등\n이무진'

## 1.2. Text Crawling with for loop
위에서 찾은 Xpath의 규칙을 바탕으로 for loop 만들자

In [11]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
# 웹페이지가 로딩될때까지 기다리고 10초가 넘어가면 웹페이지가 로딩이 됐던 안됐던 다음 명령어를 실행하겠다는 의미
driver.implicitly_wait(10)

# 멜론사이트 주소
url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []

# title crawling (TOP 50)
#  f0r문 사용하여 순위 50까지의 노래와 가수 이름 크롤링하기
for i in range(1, 51):
    title = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f"//*[@id='frm']/div/table/tbody/tr[{i}]/td[6]/div/div")))
    title_list.append(title.text) # 크롤링한 노래제목과 가수이름 ,리스트에 추가하기
    
print(title_list)

['STAY\nThe Kid LAROI, Justin Bieber', '신호등\n이무진', 'My Universe\nColdplay, 방탄소년단', 'Permission to Dance\n방탄소년단', 'OHAYO MY NIGHT\n디핵 (D-Hack), PATEKO (파테코)', 'Next Level\naespa', '바라만 본다\nMSG워너비(M.O.M)', 'Butter\n방탄소년단', 'Weekend\n태연 (TAEYEON)', '낙하 (with 아이유)\nAKMU (악뮤)', 'Dynamite\n방탄소년단', '좋아좋아\n조정석', 'Peaches (Feat. Daniel Caesar & Giveon)\nJustin Bieber', 'DUMB DUMB\n전소미', 'Queendom\nRed Velvet (레드벨벳)', 'Bad Habits\nEd Sheeran', '헤픈 우연\n헤이즈 (Heize)', '다정히 내 이름을 부르면\n경서예지, 전건호', '시간을 거슬러 (낮에 뜨는 달 X 케이윌)\n케이윌', '가을 타나 봐\n이무진', '이제 나만 믿어요\n임영웅', '비와 당신\n이무진', "롤린 (Rollin')\n브레이브걸스", 'Dun Dun Dance\n오마이걸 (OH MY GIRL)', '라일락\n아이유', 'Sticker\nNCT 127', 'Savage Love (Laxed - Siren Beat) (BTS Remix)\nJawsh 685, Jason Derulo, 방탄소년단', 'Celebrity\n아이유', 'ASAP\nSTAYC(스테이씨)', '고백\n멜로망스', '비가 오는 날엔 (2021)\n헤이즈 (Heize)', '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)\n방탄소년단', '사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)\n호미들', '가을 우체국 앞에서\n김대명', '별빛 같은 나의 사랑아\n임영웅', '색안경 (STEREOTYPE)\nSTAYC(스테이씨)'

나중에 필요한 변수(제목, 가수, 가사... 등)을 모두 긁어 한번에 데이터프레임으로 저장하여 보관한다!

## 1.3. Text Crawling (Click & Back)
클릭하고 나오기 -> 동적 크롤링 가능 (가사 크롤링 가능)

노래 제목에 링크가 걸려있기 때문에, 해당 링크까지의 XPath를 추가한다.

In [14]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

# 멜론 사이트 주소
url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 1번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[5]/div/a')))
click_element.click()    

# back
driver.back()


# 2번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[2]/td[5]/div/a')))
click_element.click()    

# back
driver.back()

## 1.4. Text Crawling including contents
- 1.2처럼 for문과 함께 써보자! (첫 페이지 5개의 글에 대해 title, artist, heart(하트 갯수), lyrics(가사)를 크롤링

- 1.3에서 사용한 click & back을 활용하자

In [15]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

# 멜론 사이트 주소
url = "https://www.melon.com/chart/index.htm"
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수 생성하기
title_list = []
artist_list = []
heart_list = []
lyrics_list = []

# crawling (TOP 5)
# 멜론 차트에 5순위까지 for문을 사용하여 크롤링하기
for i in range(1, 6):
    # click
    click_element = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{i}]/td[5]/div/a')))
    click_element.click() 

    # title crawling(노래 제목 크롤링하기)
    title = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.song_name")))
    title_list.append(title.text) # 리스트에 크롤링한 거 추가하기

    # artist crawling(가수 이름 크롤링하기)
    artist = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)")))
    artist_list.append(artist.text) # 리스트에 크롤링한 거 추가하기
    
    # heart crawling(좋아요 수 크롤링하기)
    heart = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_like_count")))
    heart_list.append(heart.text) # 리스트에 크롤링한 거 추가하기

    # lyrics crawling(노래 가사 크롤링하기)
    lyrics = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_video_summary")))
    lyrics_list.append(lyrics.text) # 리스트에 크롤링한 거 추가하기
    
    # back
    driver.back()
    
print(title_list)
print(artist_list)
print(heart_list)
print(lyrics_list)

['STAY', '신호등', 'My Universe', 'Permission to Dance', 'OHAYO MY NIGHT']
['The Kid LAROI', '이무진', 'Coldplay', '방탄소년단', '디핵 (D-Hack)']
['151,128', '214,515', '60,927', '175,881', '132,518']
["I do the same thing I told you\nthat I never would\nI told you I'd change\neven when I knew I never could\nI know that I can't find\nnobody else\nas good as you\nI need you to stay\nneed you to stay hey Oh\nI get drunk wake up\nI'm wasted still\nI realize the time\nthat I wasted here\nI feel like you can't\nfeel the way I feel\nOh I'll be fucked up\nif you can't be right here\nOh ooh-woah\nOh ooh-woah ooh-woah\nOh ooh-woah\nOh ooh-woah ooh-woah\nOh ooh-woah\nOh ooh-woah ooh-woah\nOh I'll be fucked up\nif you can't be right here\nI do the same thing I told you\nthat I never would\nI told you I'd change\neven when I knew I never could\nI know that I can't find\nnobody else\nas good as you\nI need you to stay\nneed you to stay hey\nI do the same thing I told you\nthat I never would\nI told you I'd chan

## TIP: 보통은 결과값을 데이터프레임 형태로 저장한다

In [31]:
# 결과 변수
# 리스트를 모두 모아 딕셔너리 생성
raw_result = {'title': title_list,
              'artist': artist_list,
              'heart': heart_list,
          'lyrics': lyrics_list}

# 생성한 딕셔너리를 데이터프레임으로 변환하기
result = pd.DataFrame(raw_result)

# # csv 파일로 save
# result.to_csv("MelonTop5", mode='w')

# driver 종료
driver.quit()

In [17]:
result

,title,artist,heart,lyrics
0,STAY,The Kid LAROI,"151,128",I do the same thing I told you\nthat I never w...
1,신호등,이무진,"214,515",이제야 목적지를 정했지만\n가려한 날 막아서네 난 갈 길이 먼데\n새빨간 얼굴로 화...
2,My Universe,Coldplay,"60,927","You, you are my universe and\nI just want to p..."
3,Permission to Dance,방탄소년단,"175,881",It’s the thought of being young\nWhen your hea...
4,OHAYO MY NIGHT,디핵 (D-Hack),"132,518",너를 사랑하고 있어\n너를 사랑하고 있어\n자기야 날 사랑해주면 안 될까\n말처럼 ...


데이터프레임 형식을 이용하면, 가독성도 좋고, 나중에 데이터 핸들링하기에도 편리하다!

# 2. Image Crawling

이미지 크롤링하기

- 1번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[1]/td[4]/div/a/img
- 2번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[2]/td[4]/div/a/img
...

- 50번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[50]/td[4]/div/a/img

**STEP1. URL Crawling**

In [22]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

# 멜론 차트 주소
url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
link_list = []

# # img crawling (TOP 50)
# 멜론차트 순위 50위까지의 이미지 크롤링하기
for i in range(1, 51):
    
    img = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.CSS_SELECTOR, f"#frm > div > table > tbody > tr:nth-child({i}) > td:nth-child(4) > div > a > img")))

    link_list.append(img.get_attribute('src')) # 리스트에 크롤링한 이미지 주소 추가

print(link_list)

['https://cdnimg.melon.co.kr/cm2/album/images/106/46/395/10646395_20210707141710_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/07/796/10607796_20210513201807_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/107/20/913/10720913_20210923173742_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/48/182/10648182_20210709104950_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/47/520/10447520_20200619123343_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/09/232/10609232_20210517155130_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/38/275/10638275_20210625172521_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quali

**STEP2. Download images using URLs**

자신의 디렉토리에 img 폴더 생성하고 실행

In [28]:
import urllib.request

count = 0
# 앞에서 저장한 이미지 주소들을 for을 이용하여 자신의 디렉토리에 생성하여 저장하기
for link in link_list:
    count += 1
    dir='C:/Users/chaeyeon/capstone_python/image_all' # 자신의 이미지 폴더 주소
    # 이미지 url 다운로드 하기: urllib.request.urlretrieve(image_url, image_path)
    urllib.request.urlretrieve(link, dir + '/img' + str(count) + '.jpg') 